In [ ]:
from fastai.text import *
from fastai import *
from numpy.random import seed
seed(0)
import numpy as np
import pickle
import keras
from keras.models import Model, load_model
from keras.layers import Masking, Dense, concatenate, multiply, subtract, Dropout, Embedding, LSTM, GRU, Bidirectional, GlobalMaxPooling1D, Input, TimeDistributed
from keras.optimizers import Adam

from my_layers import SelfAttLayer, weightedAccCallback
from score import score_submission, print_confusion_matrix, report_score
from eventregistry import *
from feature_engineering import polarity_features, refuting_features, word_overlap_features, hand_features
from nltk.translate.bleu_score import sentence_bleu
from rouge import Rouge
from nltk.tokenize import sent_tokenize, word_tokenize
import unidecode
from keras.preprocessing import sequence
import re
from keras.models import load_model

abbr_list = ["n't","'d","'ll","'s","'m","'ve","'re"]


In [ ]:
learn_classify = load_learner(Path('Fake-News'), 'final-6Ji.pkl') #Load the weights of the linguistics model


In [ ]:
#Defining the stance detection model 

def closest_word(originalWord, embeddings):
    words = list(embeddings.keys())
    currentClosest = words[0]
    for word in words:
        if jellyfish.jaro_winkler(originalWord, word) > jellyfish.jaro_winkler(originalWord, currentClosest):
            currentClosest = word
    print("Closest word to " + originalWord +" is " + currentClosest)
    return embeddings[currentClosest]

def remove_parenthesis(sent):
    return ' '.join(sent.replace('(', ' ').replace(')', ' ').replace('.', '').split()).lower()

def clean(s):
    # Cleans a string: Lowercasing, trimming, removing non-alphanumeric
    return " ".join(re.findall(r'\w+', s, flags=re.UNICODE)).lower()
def clean_fnc(s):
    s = unidecode.unidecode(s) # for correct tokenization
    tokens = word_tokenize(s)
    for i, tok in enumerate(tokens):
        if tok not in abbr_list:
            tokens[i] = clean(tok)
    return ' '.join(list(filter(lambda x: x != '', tokens))).lower()


#/usr/bin/env python
# -*- coding: UTF-8 -*-

# imports #
from numpy.random import seed
seed(0)
from keras.callbacks import ModelCheckpoint, EarlyStopping

import numpy as np
import pickle
import keras
from keras.models import Model, load_model
from keras.layers import Masking, Dense, concatenate, multiply, subtract, Dropout, Embedding, LSTM, GRU, Bidirectional, GlobalMaxPooling1D, Input, TimeDistributed
from keras.optimizers import Adam

from my_layers import SelfAttLayer, weightedAccCallback
from score import score_submission, print_confusion_matrix, report_score

# Some hyperparameters #
hidden_units = 300
max_seq_len = 50
max_seqs = 30
########################

print("Opening features")
with open('Data_Fake/features.pkl', 'rb') as inpFeat:
    overlapFeatures_fnc = pickle.load(inpFeat)
    refutingFeatures_fnc = pickle.load(inpFeat)
    polarityFeatures_fnc = pickle.load(inpFeat)
    handFeatures_fnc = pickle.load(inpFeat)
    overlapFeatures_fnc_test = pickle.load(inpFeat)
    refutingFeatures_fnc_test = pickle.load(inpFeat)
    polarityFeatures_fnc_test = pickle.load(inpFeat)
    handFeatures_fnc_test = pickle.load(inpFeat)
    overlapFeatures_nli = pickle.load(inpFeat)
    refutingFeatures_nli = pickle.load(inpFeat)
    polarityFeatures_nli = pickle.load(inpFeat)
    handFeatures_nli = pickle.load(inpFeat)
    overlapFeatures_nli_test = pickle.load(inpFeat)
    refutingFeatures_nli_test = pickle.load(inpFeat)
    polarityFeatures_nli_test = pickle.load(inpFeat)
    handFeatures_nli_test = pickle.load(inpFeat)
    overlapFeatures_matched_test = pickle.load(inpFeat)
    refutingFeatures_matched_test = pickle.load(inpFeat)
    polarityFeatures_matched_test = pickle.load(inpFeat)
    handFeatures_matched_test = pickle.load(inpFeat)
    overlapFeatures_mismatched_test = pickle.load(inpFeat)
    refutingFeatures_mismatched_test = pickle.load(inpFeat)
    polarityFeatures_mismatched_test = pickle.load(inpFeat)
    handFeatures_mismatched_test = pickle.load(inpFeat)
    overlapFeatures_fnc_two = pickle.load(inpFeat)
    refutingFeatures_fnc_two = pickle.load(inpFeat)
    polarityFeatures_fnc_two = pickle.load(inpFeat)
    handFeatures_fnc_two = pickle.load(inpFeat)
    overlapFeatures_fnc_two_test = pickle.load(inpFeat)
    refutingFeatures_fnc_two_test = pickle.load(inpFeat)
    polarityFeatures_fnc_two_test = pickle.load(inpFeat)
    handFeatures_fnc_two_test = pickle.load(inpFeat)
    bleu_nli = pickle.load(inpFeat)
    bleu_nli_test = pickle.load(inpFeat)
    bleu_matched = pickle.load(inpFeat)
    bleu_mismatched = pickle.load(inpFeat)
    rouge_nli = pickle.load(inpFeat)
    rouge_nli_test = pickle.load(inpFeat)
    rouge_matched = pickle.load(inpFeat)
    rouge_mismatched = pickle.load(inpFeat)
    bleu_fnc = pickle.load(inpFeat)
    bleu_fnc_test = pickle.load(inpFeat)
    bleu_two_sentences = pickle.load(inpFeat)
    bleu_two_sentences_test = pickle.load(inpFeat)
    rouge_fnc = pickle.load(inpFeat)
    rouge_fnc_test = pickle.load(inpFeat)
    rouge_two_sentences = pickle.load(inpFeat)
    rouge_two_sentences_test = pickle.load(inpFeat)

del overlapFeatures_nli, refutingFeatures_nli, polarityFeatures_nli, handFeatures_nli, overlapFeatures_nli_test , refutingFeatures_nli_test, \
    polarityFeatures_nli_test, handFeatures_nli_test, overlapFeatures_matched_test, refutingFeatures_matched_test, polarityFeatures_matched_test, \
    handFeatures_matched_test, overlapFeatures_mismatched_test, refutingFeatures_mismatched_test, polarityFeatures_mismatched_test, \
    handFeatures_mismatched_test, bleu_nli, bleu_nli_test, bleu_matched, bleu_mismatched, rouge_nli, rouge_nli_test, rouge_matched, rouge_mismatched

print("Opening variables")
with open('Data_Fake/variables.pkl', 'rb') as inp:
    embedding_weights = pickle.load(inp)
    X1 = pickle.load(inp)
    X2 = pickle.load(inp)
    Y = pickle.load(inp)
    X1_test = pickle.load(inp)
    X2_test = pickle.load(inp)
    Y_test = pickle.load(inp)
    X1_nli = pickle.load(inp)
    X2_nli = pickle.load(inp)
    Y_nli = pickle.load(inp)
    X1_test_nli = pickle.load(inp)
    X2_test_nli = pickle.load(inp)
    Y_test_nli = pickle.load(inp)
    X1_test_matched = pickle.load(inp)
    X2_test_matched = pickle.load(inp)
    Y_test_matched = pickle.load(inp)
    X1_test_mismatched = pickle.load(inp)
    X2_test_mismatched = pickle.load(inp)
    Y_test_mismatched = pickle.load(inp)
    X2_two_sentences = pickle.load(inp)
    X2_test_two_sentences = pickle.load(inp)
    tokenizer = pickle.load(inp)

    
del X1_nli, X2_nli, Y_nli, X1_test_nli, X2_test_nli, Y_test_nli, X1_test_matched, X2_test_matched, Y_test_matched, X1_test_mismatched, \
    X2_test_mismatched, Y_test_mismatched

print("Opening similarities")
with open('Data_Fake/similarity.pkl', 'rb') as inpSim:
    cosFeatures = pickle.load(inpSim)
    cosFeatures_test = pickle.load(inpSim)
    cosFeatures_nli = pickle.load(inpSim)
    cosFeatures_nli_test = pickle.load(inpSim)
    cosFeatures_matched = pickle.load(inpSim)
    cosFeatures_mismatched = pickle.load(inpSim)

cosFeatures = np.array(cosFeatures)
cosFeatures_test = np.array(cosFeatures_test)

cosFeatures_fnc = []
cosFeatures_two = []
for feat in cosFeatures:
    cosFeatures_fnc += [feat[0]]
    cosFeatures_two += [feat[1]]
cosFeatures_fnc = np.array(cosFeatures_fnc)
cosFeatures_two = np.array(cosFeatures_two)

cosFeatures_fnc_test = []
cosFeatures_two_test = []
for feat in cosFeatures_test:
    cosFeatures_fnc_test += [feat[0]]
    cosFeatures_two_test += [feat[1]]
cosFeatures_fnc_test = np.array(cosFeatures_fnc_test)
cosFeatures_two_test = np.array(cosFeatures_two_test)

del cosFeatures_nli, cosFeatures_nli_test, cosFeatures_matched, cosFeatures_mismatched

with open("Data_Fake/cider_fnc.pkl", "rb") as ciderFile:
    cider_fnc_train = pickle.load(ciderFile, encoding='latin1')
    cider_fnc_test = pickle.load(ciderFile, encoding='latin1')
    cider_two_train = pickle.load(ciderFile, encoding='latin1')
    cider_two_test = pickle.load(ciderFile, encoding='latin1')

import pickle as cPickle

with open("talos-fnc-1-py3/tree_model/train.basic.pkl", "rb") as countsTrain:
    names = cPickle.load(countsTrain)
    talos_counts_train = cPickle.load(countsTrain, encoding='latin1')
print('Done')
with open("talos-fnc-1-py3/tree_model/test.basic.pkl", "rb") as countsTest:
    names = cPickle.load(countsTest)
    talos_counts_test = cPickle.load(countsTest, encoding='latin1')

with open("talos-fnc-1-py3/tree_model/train.sim.tfidf.pkl", "rb") as tfidfSim_train:
    talos_tfidfsim_train = cPickle.load(tfidfSim_train, encoding='latin1')

with open("talos-fnc-1-py3/tree_model/test.sim.tfidf.pkl", "rb") as tfidfSim_test:
    talos_tfidfsim_test = cPickle.load(tfidfSim_test, encoding='latin1')

with open("talos-fnc-1-py3/tree_model/train.headline.svd.pkl", "rb") as svdHealine_train:
    talos_svdHeadline_train = cPickle.load(svdHealine_train, encoding='latin1')

with open("talos-fnc-1-py3/tree_model/test.headline.svd.pkl", "rb") as svdHealine_test:
    talos_svdHeadline_test = cPickle.load(svdHealine_test, encoding='latin1')

with open("talos-fnc-1-py3/tree_model/train.body.svd.pkl", "rb") as svdBody_train:
    talos_svdBody_train = cPickle.load(svdBody_train, encoding='latin1')

with open("talos-fnc-1-py3/tree_model/test.body.svd.pkl", "rb") as svdBody_test:
    talos_svdBody_test = cPickle.load(svdBody_test, encoding='latin1')

with open("talos-fnc-1-py3/tree_model/train.sim.svd.pkl", "rb") as svdSim_train:
    talos_svdsim_train = cPickle.load(svdSim_train, encoding='latin1')

with open("talos-fnc-1-py3/tree_model/test.sim.svd.pkl", "rb") as svdSim_test:
    talos_svdsim_test = cPickle.load(svdSim_test, encoding='latin1')

with open("talos-fnc-1-py3/tree_model/train.headline.word2vec.pkl", "rb") as w2vHealine_train:
    talos_w2vHeadline_train = cPickle.load(w2vHealine_train, encoding='latin1')

with open("talos-fnc-1-py3/tree_model/test.headline.word2vec.pkl", "rb") as w2vHealine_test:
    talos_w2vHeadline_test = cPickle.load(w2vHealine_test, encoding='latin1')

with open("talos-fnc-1-py3/tree_model/train.body.word2vec.pkl", "rb") as w2vBody_train:
    talos_w2vBody_train = cPickle.load(w2vBody_train, encoding='latin1')

with open("talos-fnc-1-py3/tree_model/test.body.word2vec.pkl", "rb") as w2vBody_test:
    talos_w2vBody_test = cPickle.load(w2vBody_test, encoding='latin1')

with open("talos-fnc-1-py3/tree_model/train.sim.word2vec.pkl", "rb") as w2vSim_train:
    talos_w2vsim_train = cPickle.load(w2vSim_train, encoding='latin1')

with open("talos-fnc-1-py3/tree_model/test.sim.word2vec.pkl", "rb") as w2vSim_test:
    talos_w2vsim_test = cPickle.load(w2vSim_test, encoding='latin1')

with open("talos-fnc-1-py3/tree_model/train.headline.senti.pkl", "rb") as sentiHealine_train:
    talos_sentiHeadline_train = cPickle.load(sentiHealine_train, encoding='latin1')

with open("talos-fnc-1-py3/tree_model/test.headline.senti.pkl", "rb") as sentiHealine_test:
    talos_sentiHeadline_test = cPickle.load(sentiHealine_test, encoding='latin1')

with open("talos-fnc-1-py3/tree_model/train.body.senti.pkl", "rb") as sentiBody_train:
    talos_sentiBody_train = cPickle.load(sentiBody_train, encoding='latin1')

with open("talos-fnc-1-py3/tree_model/test.body.senti.pkl", "rb") as sentiBody_test:
    talos_sentiBody_test = cPickle.load(sentiBody_test, encoding='latin1')

########################## Definir o modelo ##################################### 

#Define some model layers #
print('Done ! Done ! Done !')
early_stop = EarlyStopping(monitor='loss', patience=2, verbose=1, restore_best_weights=True)
weightedAccuracy = weightedAccCallback(X1_test, X2_test, Y_test, overlapFeatures_fnc_test, refutingFeatures_fnc_test, polarityFeatures_fnc_test, handFeatures_fnc_test,  \
                                       cosFeatures_fnc_test,cosFeatures_two_test, bleu_fnc_test, rouge_fnc_test,cider_fnc_test, X2_test_two_sentences, overlapFeatures_fnc_two_test, \
                                       refutingFeatures_fnc_two_test, polarityFeatures_fnc_two_test, handFeatures_fnc_two_test, \
                                       bleu_two_sentences_test, rouge_two_sentences_test, cider_two_test
                                       , talos_counts_test, talos_tfidfsim_test, talos_svdHeadline_test, \
                                       talos_svdBody_test, talos_svdsim_test,talos_w2vHeadline_test, talos_w2vBody_test, talos_w2vsim_test, talos_sentiHeadline_test, talos_sentiBody_test)
                                     

embedding_layer = Embedding( embedding_weights.shape[0], embedding_weights.shape[1], input_length=max_seq_len, weights=[embedding_weights], trainable=False )
lstm1 = LSTM(hidden_units, implementation=2, return_sequences=True, name='lstm1' )
lstm1 = Bidirectional(lstm1, name='bilstm1')
right_branch_lstm1 = LSTM(hidden_units, implementation=2, return_sequences=True )
right_branch_lstm1 = Bidirectional(right_branch_lstm1)

#####################################

# Define the inputs for the model #

input_headline = Input(shape=(max_seq_len,))
input_two = Input(shape=(max_seq_len,))
input_body = Input(shape=(max_seqs, max_seq_len,))
input_overlap = Input(shape=(1,))
input_overlap_two = Input(shape=(1,))
input_refuting = Input(shape=(15,))
input_refuting_two = Input(shape=(15,))
input_polarity = Input(shape=(2,))
input_polarity_two = Input(shape=(2,))
input_hand = Input(shape=(26,))
input_hand_two = Input(shape=(26,))
input_sim = Input(shape=(1,))
input_sim_two = Input(shape=(1,))
input_bleu = Input(shape=(1,))
input_bleu_two = Input(shape=(1,))
input_rouge = Input(shape=(3,))
input_rouge_two = Input(shape=(3,))
input_cider = Input(shape=(1,))
input_cider_two = Input(shape=(1,))

input_talos_count = Input(shape=(41,))
input_talos_tfidfsim = Input(shape=(1,))
input_talos_headline_svd = Input(shape=(50,))
input_talos_body_svd = Input(shape=(50,))
input_talos_svdsim = Input(shape=(1,))
input_talos_headline_w2v = Input(shape=(300,))
input_talos_body_w2v = Input(shape=(300,))
input_talos_w2vsim = Input(shape=(1,))
input_talos_headline_senti = Input(shape=(4,))
input_talos_body_senti = Input(shape=(4,))


###############################

# Define the sentence encoder #

mask = Masking(mask_value=0, input_shape=(max_seq_len,))(input_headline)
embed = embedding_layer(mask)
l1 = lstm1(embed)
drop1 = Dropout(0.1)(l1)
maxim = GlobalMaxPooling1D()(drop1)
att = SelfAttLayer(name='attention')(drop1)
out = concatenate([maxim, att])
HeadlineEncoder = Model(input_headline, maxim, name='HeadlineEncoder')

# HeadlineEncoder.set_weights(layer_dict['SentenceEncoder'].get_weights())

##############################

# Define the document encoder #

body_sentence = TimeDistributed(HeadlineEncoder)(input_body)
body_g1 = right_branch_lstm1(body_sentence)
body_g1 = Dropout(0.1)(body_g1)
body_maxim = GlobalMaxPooling1D()(body_g1)
body_att = SelfAttLayer()(body_g1)
body_out = concatenate([body_maxim, body_att])
DocumentEncoder = Model(input_body, body_maxim, name='DocumentEncoder')

##############################

# Combining both representations #

headline_representation = HeadlineEncoder(input_headline)
document_representation = DocumentEncoder(input_body)

# Match between headline and first two sentences from body #

two_sentences_representation = HeadlineEncoder(input_two)
concat_two = concatenate([headline_representation, two_sentences_representation])
mul_two = multiply([headline_representation, two_sentences_representation])
dif_two = subtract([headline_representation, two_sentences_representation])
final_merge_two = concatenate([concat_two, mul_two, dif_two, input_overlap_two, input_refuting_two, input_polarity_two, input_hand_two, \
                               input_sim_two, input_bleu_two, input_rouge_two, input_cider_two])
drop3_two = Dropout(0.1)(final_merge_two)
dense1_two = Dense(hidden_units*2, activation='relu')(drop3_two)
# , weights=layer_dict['dense1'].get_weights()
drop4_two = Dropout(0.1)(dense1_two)
dense2_two = Dense(hidden_units, activation='relu')(drop4_two)
# ,weights=layer_dict['dense2'].get_weights()
match = Dropout(0.1)(dense2_two)

#####################################################

concat = concatenate([headline_representation, document_representation])
mul = multiply([headline_representation, document_representation])
dif = subtract([headline_representation, document_representation])
final_merge = concatenate([concat, mul, dif, input_overlap, input_refuting, input_polarity, input_hand, input_sim, input_bleu, input_rouge, input_cider])
drop3 = Dropout(0.1)(final_merge)
dense1 = Dense(hidden_units*2, activation='relu', name='dense1')(drop3)
# , weights=layer_dict['dense1'].get_weights()
drop4 = Dropout(0.1)(dense1)
dense2 = Dense(hidden_units, activation='relu', name='dense2')(drop4)
# , weights=layer_dict['dense2'].get_weights()
drop5 = Dropout(0.1)(dense2)
concat_final = concatenate([drop5,match,input_talos_count, input_talos_tfidfsim, input_talos_headline_svd, input_talos_body_svd, \
                     input_talos_svdsim, input_talos_headline_w2v, input_talos_body_w2v, input_talos_w2vsim, \
                     input_talos_headline_senti, input_talos_body_senti])
drop6 = Dropout(0.1)(concat_final)
dense3 = Dense(4, activation='softmax')(drop6)
final_model = Model([input_headline, input_body,input_overlap, input_refuting, input_polarity, input_hand, \
                     input_sim, input_sim_two, input_bleu, input_rouge,input_cider, input_two, input_overlap_two, input_refuting_two, input_polarity_two, input_hand_two, \
                     input_bleu_two, input_rouge_two, input_cider_two, input_talos_count, input_talos_tfidfsim, input_talos_headline_svd, input_talos_body_svd, \
                     input_talos_svdsim, input_talos_headline_w2v, input_talos_body_w2v, input_talos_w2vsim, \
                     input_talos_headline_senti, input_talos_body_senti], dense3)
#######################################################################################

final_model.summary()

In [ ]:
#Extract Articles 

keyss = ['Iran' , 'Olympic']
def get_articles(keyss):
  er = EventRegistry(apiKey = 'ENTER API KEY')
  q = QueryArticlesIter(
      keywords = QueryItems.AND(keyss),
   dataType   = ["news", "blog"],
       lang = 'eng',
       startSourceRankPercentile = 0,
      endSourceRankPercentile = 30)
  # obtain at most 500 newest articles or blog posts
  all_data=[]
  for art in q.execQuery(er, sortBy = "relevancy", maxItems = 40):
      all_data.append(art)
  return(all_data)

all_articles = get_articles(keyss)
len(all_articles)

In [ ]:
head = 'Iran’s Only Female Olympic Medalist Defects Over ‘Lies’ and ‘Injustice’'
# bodyy = """Elon Musk kicked off a Tuesday presentation at Tesla’s Shanghai plant by tearing off his blazer and dancing enthusiastically to a song by his girlfriend, pop star Grimes. The eccentric CEO and founder has a multibillion-dollar reason to be in such high spirits. Tesla stock has spiked since the company revealed last week that it delivered a record 112,000 vehicles in the fourth quarter, modestly exceeding the consensus estimate among analysts, and announced that its first full-scale production plant outside the U.S.—in Shanghai, China—is now delivering vehicles. Shares surged 13% in the week through January 9—boosting Musk’s net worth by $2.1 billion, to $28.8 billion. The 49-year-old billionaire is currently the 34th-richest person in the world. Musk owns nearly 22% of Tesla’s stock, which accounts for roughly half of his fortune. He also has an estimated $12.5 billion stake in SpaceX, his privately held aerospace company known for reusable rockets. Today In: Billionaires CHINA-AUTOMOBILE-MANUFACTURING-TESLA-US Musk dances to Grimes' "Oblivion" at Tesla's Shanghai plant.AFP VIA GETTY IMAGES On Tuesday, Tesla’s market cap hit $83 billion, making it the most valuable car company in American history (not accounting for inflation), besting Ford’s market value of $81 billion in 1999, according to data from Dow Jones. By Friday’s market close, Tesla’s market cap had crept all the way up to $85.8 billion. That is close behind the $87 billion combined market cap of GM and Ford. PROMOTED Tesla stock has doubled since October, but there have been road bumps along the way. In November, Musk’s net worth fell by $768 million in a single day after the botched unveiling of the Cybertruck sunk Tesla’s share price by 6%. And Wall Street analysts remain highly divided on the company. Out of a group of 36 analysts tracked by Bloomberg, 15 have issued sell ratings, 11 have issued holds and 10 have issued buys. Wedbush analyst Daniel Ives is confident in Tesla, highlighting growth potential in China. “We believe Tesla’s ability to ramp production and demand in the key China region during the course of 2020 will be a major swing factor on the stock,” Ives wrote in a research note. JPMorgan analyst Ryan Brinkman maintained his sell rating. Though he raised his price target, Brinkman noted that deliveries for the more expensive Models S and X (versus the Model 3) have fallen 30% year over year. “We remain underweight-rated on TSLA shares, however, on what we see as a lofty valuation (an $80 billion market cap after its recent run for a company about to report its tenth consecutive annual net loss) coupled with high investor expectations and high execution risk,” Brinkman wrote. CFRA analyst Garrett Nelson, meanwhile, downgraded his rating from hold to sell. “In our view, TSLA shares now appear fully valued after a meteoric run-up since bottoming at $177 a share last June and are not reflecting various near-term risks,” Nelson wrote. “With TSLA’s market cap now exceeding GM and Ford combined despite having only about 3% of joint vehicle sales volume, we think investors have given TSLA plenty of credit for future growth, raising execution risk.” Heavenly Bodies: Fashion & The Catholic Imagination Costume Institute Gala - Cocktails Musk and Canadian pop star Grimes attend the 2018 Met Gala.GETTY IMAGES FOR VOGUE It has been a busy week for Musk. His girlfriend, Grimes, has posted two pregnancy photos on Instagram. Musk has yet to formally confirm he is the father, but in response to a tweeted video of him dancing to Grimes’ song, “Oblivion,” he cheekily replied, “Haha spectrum af” with a baby emoji. He also commented on Grimes’ tweet of a pregnancy photo with “x is y.” If Tesla’s run-up continues, its share price may double again by the time Musk’s seventh child is born—adding billions to his already massive net worth. """
all_body = []
for i in all_articles:
  all_body.append(i['body'])

all_head = []
for i in range(0,len(all_body)):
  all_head.append(head)

In [ ]:
X1_head = all_head
X2_body = all_body

In [ ]:

rouge = Rouge()

overlapFeatures_fnc_test = np.array(word_overlap_features(X1_head, X2_body))
refutingFeatures_fnc_test = np.array(refuting_features(X1_head, X2_body))
polarityFeatures_fnc_test = np.array(polarity_features(X1_head, X2_body))
handFeatures_fnc_test = np.array(hand_features(X1_head, X2_body))

bleu_fnc_test = []
for i in range(len(X1_head)):
    split_doc = sent_tokenize(X1_head[i])
    for j in range(len(split_doc)):
        split_doc[j] = word_tokenize(clean_fnc(split_doc[j]))
    bleu_fnc_test += [ sentence_bleu(split_doc, word_tokenize(X1_head[i]) ) ]
bleu_fnc_test = np.array(bleu_fnc_test)

rouge_fnc_test = []
fails = 0
for i in range(len(X1_head)):
    rouge_values = []
    try:
        scores = rouge.get_scores(clean(X2_test[i]), clean(X1_head[i]))
        rouge_values += [scores[0]['rouge-1']['f']]
        rouge_values += [scores[0]['rouge-2']['f']]
        rouge_values += [scores[0]['rouge-l']['f']]
    except:
        fails += 1
        rouge_values = [0,0,0]
    rouge_fnc_test += [rouge_values]
print("ROUGE FNC TEST: {} fails".format(fails))
rouge_fnc_test = np.array(rouge_fnc_test)

X2_test_two_sentences = []
for document in X2_body:
    sentences = sent_tokenize(document)
    try:
        X2_test_two_sentences += ['| ' + clean_fnc(sentences[0]) + ' ' + clean_fnc(sentences[1]) + ' |']
    except:
        X2_test_two_sentences += ['| ' + clean_fnc(sentences[0]) + ' |']
  

overlapFeatures_fnc_two_test = np.array(word_overlap_features(X1_head, X2_test_two_sentences))
refutingFeatures_fnc_two_test = np.array(refuting_features(X1_head, X2_test_two_sentences))
polarityFeatures_fnc_two_test = np.array(polarity_features(X1_head, X2_test_two_sentences))
handFeatures_fnc_two_test = np.array(hand_features(X1_head, X2_test_two_sentences))


bleu_two_sentences_test = []
for i in range(len(X1_head)):
    bleu_two_sentences_test += [ sentence_bleu(word_tokenize(clean_fnc(X2_test_two_sentences[i])), \
                                          word_tokenize(X1_head[i]) ) ]
bleu_two_sentences_test = np.array(bleu_two_sentences_test)



rouge_two_sentences_test = []
for i in range(len(X1_head)):
    rouge_values = []
    scores = rouge.get_scores(clean(X2_test_two_sentences[i]), clean(X1_head[i]))
    rouge_values += [scores[0]['rouge-1']['f']]
    rouge_values += [scores[0]['rouge-2']['f']]
    rouge_values += [scores[0]['rouge-l']['f']]
    rouge_two_sentences_test += [rouge_values]
rouge_two_sentences_test = np.array(rouge_two_sentences_test)

X2_test_two_sentences = sequence.pad_sequences( tokenizer.texts_to_sequences( X2_test_two_sentences ) , maxlen=max_seq_len )

print('Done')

In [ ]:

X1_head = sequence.pad_sequences( tokenizer.texts_to_sequences( X1_head) , maxlen=max_seq_len ) # Mesma coisa para headlines de teste
X1_head = np.asarray(X1_head)

data_aux = np.zeros( ( len(X2_body) , max_seqs , max_seq_len ) ) # len(X2) = numero total de bodies do dataset, max_seq_len = 30, max_seqs = 15
for i, sentences in enumerate(X2_body):
    sentences = sent_tokenize( sentences )
    sentences = list(map(lambda x: '| ' + clean_fnc(x) + ' |', sentences))
    aux = [ ]
    for j, sent in enumerate(sentences):
        if j < max_seqs: data_aux[i,j] = sequence.pad_sequences( tokenizer.texts_to_sequences( [ sent ] ) , maxlen=max_seq_len )[0]
X2_body = np.asarray(data_aux)

# final_model.load_weights("fnc-weights.h5")
final_model1 = load_model("fnc-weights-save.h5")

test_outputs = []
test_predictions = []
labels = ['unrelated', 'agree', 'disagree', 'discuss']


In [ ]:
aux = final_model1.predict([X1_head, 
                                  X2_body, overlapFeatures_fnc_test, refutingFeatures_fnc_test, polarityFeatures_fnc_test, handFeatures_fnc_test,  \
                                  cosFeatures_fnc_test, cosFeatures_two_test, bleu_fnc_test, rouge_fnc_test, cider_fnc_test, \
                                  X2_test_two_sentences, overlapFeatures_fnc_two_test, refutingFeatures_fnc_two_test, polarityFeatures_fnc_two_test, handFeatures_fnc_two_test, \
                                  bleu_two_sentences_test, rouge_two_sentences_test, cider_two_test, talos_counts_test, talos_tfidfsim_test, talos_svdHeadline_test, talos_svdBody_test, talos_svdsim_test, \
                                  talos_w2vHeadline_test, 
                               talos_w2vBody_test, 
                           talos_w2vsim_test, 
                           talos_sentiHeadline_test, 
                           talos_sentiBody_test])
# aux = final_model1.predict([X1_head, X2_body])

In [ ]:
test_predictions = []
preds = []
for prediction in aux:
    pred = prediction.argmax()
    if pred == 0:
        one_hot = [1,0,0,0]
    if pred == 1:
        one_hot = [0,1,0,0]
    if pred == 2:
        one_hot = [0,0,1,0]
    if pred == 3:
        one_hot = [0,0,0,1]
    preds += [one_hot]
    test_predictions += [labels[prediction.argmax()]]

In [ ]:
def score(test_predictions):
  score = 0
  count = 0 
  for i in test_predictions:
    if i == 'agree':
      score += 1
      
    if i == 'disagree':
      score-= 2
    if i == 'discuss':
      score -= 0
       
    count +=1
  if score > 1:
    print('REAL ', score)
  else:
    print('FAKE', score)


In [ ]:
ling = learn_classify.predict(head)
print('*' * 15)

print('Stance only')
score(test_predictions)
print('*' * 15)
print('Linguistics Only', ling)
print('*' * 15)

hh = ling[2][0]
hhh = ling[2][1] #Second Accuracy 

# if hh > hhh:
#   meta = hh

# else:
#   meta = hhh

if len(all_body) > 3 and len(keyss) > 1:
      print('Stance Detection Comined - ')
      score(test_predictions)

elif hhh < 0.7 and len(all_body) < 10:
     print('FAKE')

else:
  print('Linguisics Comined - ')
  print(learn_classify.predict(bodyy))

